In [1]:
# Import necessary libraries
from googleapiclient.discovery import build
from youtube_transcript_api import YouTubeTranscriptApi, NoTranscriptFound
import pandas as pd
import time
import random
import sys
import os

## Load datasets

In [6]:
largest = "../data/raw/video_metadata.xlsx"
df_1 = pd.read_excel(largest, engine='openpyxl')  # Existing file

video_metadata_by_category = "../data/raw/video_metadata_by_category.xlsx"
df_2 = pd.read_excel(video_metadata_by_category, engine='openpyxl')  # Existing file

video_metadata_by_category_random = "../data/raw/video_metadata_by_category_random.xlsx"
df_3 = pd.read_excel(video_metadata_by_category_random, engine='openpyxl')  # Existing file

video_metadata_by_category_random_1 = "../data/raw/video_metadata_by_category_random_1.xlsx"
df_4 = pd.read_excel(video_metadata_by_category_random_1, engine='openpyxl')  # Existing file

video_metadata_by_category_1 = "../data/raw/video_metadata_by_category_1.xlsx"
df_5 = pd.read_excel(video_metadata_by_category_1, engine='openpyxl')  # Existing file

video_metadata_by_category_kids_1 = "../data/raw/video_metadata_by_category_kids_1.xlsx"
df_6 = pd.read_excel(video_metadata_by_category_kids_1, engine='openpyxl')

video_metadata_by_category_kids_2 = "../data/raw/video_metadata_by_category_kids_2.xlsx"
df_7 = pd.read_excel(video_metadata_by_category_kids_2, engine='openpyxl')

video_metadata_by_category_kids_3 = "../data/raw/video_metadata_by_category_kids_3.xlsx"
df_8 = pd.read_excel(video_metadata_by_category_kids_3, engine='openpyxl')

video_metadata_by_category_kids = "../data/raw/video_metadata_by_category_kids.xlsx"
df_9 = pd.read_excel(video_metadata_by_category_kids, engine='openpyxl')

In [7]:
# Standardize column names in dataset2 to match dataset1 for merging
df_1 = df_1.rename(columns={
    'Video ID': 'video_id',
    'Title': 'title',
    'Description': 'description',
    'Published At': 'published_at',
    'Channel Title': 'channel_title',
    'Tags': 'tags',
    'Category ID': 'category_id',
    'Category Label': 'Category Label',
    'Default Audio Language': 'default_audio_language',
    'Transcript': 'transcript'
})

# Add missing columns in df2
df_1['duration'] = None
df_1['view_count'] = None
df_1['like_count'] = None
df_1['dislike_count'] = None

In [8]:
# Combine the datasets
combined_df = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9], ignore_index=True)

In [10]:
combined_df.head(5)

,video_id,title,description,published_at,channel_title,tags,category_id,Category Label,default_audio_language,transcript,duration,view_count,like_count,dislike_count
0,ju4KQT0wL0I,All The Stars,Provided to YouTube by Universal Music Group\n...,2018-12-12T09:55:47Z,Kendrick Lamar - Topic,"Kendrick Lamar, ケンドリックラマー, ケンドリック・ラマー, SZA, シザ...",10,Music,NaN,Transcript not available,None,None,None,None
1,7YaqzpitBXw,What are MLPs (Multilayer Perceptrons)?,Learn about watsonx: https://ibm.biz/BdvxRg\n\...,2022-07-11T12:00:07Z,IBM Technology,"IBM, IBM Cloud, AI, Artificial Intelligence, M...",27,Education,en-US,You've probably heard of AI that can do really...,None,None,None,None
2,ZQbWWOzvyfo,Verizon,Switch to Fios Home Internet w/ no hidden fees...,2024-07-03T02:03:05Z,Verizon,NaN,28,Science & Technology,en-US,[Music] my internet my rules introducing my ho...,None,None,None,None
3,u4gEBRSKi2E,Is Being Fat A Choice? Fit Women vs Fat Women ...,Got injured in an accident? You could be one ...,2024-10-16T16:01:03Z,Jubilee,"jubilee, jubilee media, jubilee project, middl...",24,Entertainment,en,if you see me you consider me obese I wouldn't...,None,None,None,None
4,hCcwCv3G1FQ,St. Chroma,Provided to YouTube by Columbia\n\nSt. Chroma ...,2024-10-28T10:02:12Z,"Tyler, The Creator - Topic","Tyler, The Creator Daniel Caesar CHROMAKOPIA S...",10,Music,NaN,Transcript not available,None,None,None,None


In [11]:
combined_df.columns

Index(['video_id', 'title', 'description', 'published_at', 'channel_title',
       'tags', 'category_id', 'Category Label', 'default_audio_language',
       'transcript', 'duration', 'view_count', 'like_count', 'dislike_count'],
      dtype='object')

In [12]:
# Sort the dataframe by 'category_id' in ascending order
combined_df = combined_df.sort_values(by='category_id')

# Reset the index to maintain a clean dataframe
combined_df.reset_index(drop=True, inplace=True)

# Display the first few rows of the sorted dataframe
combined_df.head()

,video_id,title,description,published_at,channel_title,tags,category_id,Category Label,default_audio_language,transcript,duration,view_count,like_count,dislike_count
0,R7KfBjoLGOY,Every Swear Word RANKED...,Every Swear Word RANKED... today we rank every...,2024-06-15T20:34:45Z,The Duck,NaN,1,Film & Animation,NaN,swear words a very essential part of the vocab...,None,None,None,None
1,GkQmkZNLCeM,Best Toy Learning Video for Toddlers and Kids ...,Best Toy Learning Video for Toddlers and Kids ...,2021-07-08T18:29:49Z,Genevieve's Playhouse - Learning Videos for Kids,"best, learning video for toddlers, for kids, c...",1,Film & Animation,en,Transcript not available,None,None,None,None
2,_et1i-ykdxA,Best ABC Learning Toy Video for Toddlers! Paw ...,Best ABC Learning Toy Video for Toddlers! Paw ...,2022-07-14T16:45:05Z,Genevieve's Playhouse - Learning Videos for Kids,"abc, for toddlers, for baby, learning, educati...",1,Film & Animation,en,Transcript not available,None,None,None,None
3,_UNe6tz28OI,Best Toy Learning Video for Toddlers and Kids ...,Best Toy Learning Video for Toddlers and Kids ...,2021-03-23T01:43:53Z,Genevieve's Playhouse - Learning Videos for Kids,"best, toddler learning videos, for kids, for b...",1,Film & Animation,en,Transcript not available,None,None,None,None
4,zamt_1PAwqE,The Joy of Sharing Short Episode | Fun Learni...,"Welcome to ""The Learning Show"", the exciting a...",2023-05-19T14:42:38Z,Videogyan Shows - Educational Videos For Kids,"shorts, learning for kids, educational videos,...",1,Film & Animation,en,Transcript not available,None,None,None,None


In [15]:
# Create a dictionary for category_id and Category Label, ensuring Category Label is not NaN
category_dict = (
    combined_df[['category_id', 'Category Label']]
    .dropna(subset=['Category Label'])  # Remove rows where Category Label is NaN
    .drop_duplicates()
    .set_index('category_id')['Category Label']
    .to_dict()
)

# Display the dictionary
print(category_dict)

{1: 'Film & Animation', 2: 'Autos & Vehicles', 10: 'Music', 15: 'Pets & Animals', 17: 'Sports', 19: 'Travel & Events', 20: 'Gaming', 22: 'People & Blogs', 23: 'Comedy', 24: 'Entertainment', 25: 'News & Politics', 26: 'Howto & Style', 27: 'Education', 28: 'Science & Technology', 29: 'Nonprofits & Activism'}


In [18]:
# Fill missing or empty Category Label values using the category_dict
combined_df['Category Label'] = combined_df.apply(
    lambda row: category_dict.get(row['category_id']) if pd.isna(row['Category Label']) or row['Category Label'] == '' else row['Category Label'],
    axis=1
)

# Verify the updated DataFrame
print(combined_df.head())

      video_id                                              title  \
0  R7KfBjoLGOY                         Every Swear Word RANKED...   
1  GkQmkZNLCeM  Best Toy Learning Video for Toddlers and Kids ...   
2  _et1i-ykdxA  Best ABC Learning Toy Video for Toddlers! Paw ...   
3  _UNe6tz28OI  Best Toy Learning Video for Toddlers and Kids ...   
4  zamt_1PAwqE  The Joy of Sharing Short Episode |  Fun Learni...   

                                         description          published_at  \
0  Every Swear Word RANKED... today we rank every...  2024-06-15T20:34:45Z   
1  Best Toy Learning Video for Toddlers and Kids ...  2021-07-08T18:29:49Z   
2  Best ABC Learning Toy Video for Toddlers! Paw ...  2022-07-14T16:45:05Z   
3  Best Toy Learning Video for Toddlers and Kids ...  2021-03-23T01:43:53Z   
4  Welcome to "The Learning Show", the exciting a...  2023-05-19T14:42:38Z   

                                      channel_title  \
0                                          The Duck   
1  Gen

In [22]:
combined_df

,video_id,title,description,published_at,channel_title,tags,category_id,Category Label,default_audio_language,transcript,duration,view_count,like_count,dislike_count
0,R7KfBjoLGOY,Every Swear Word RANKED...,Every Swear Word RANKED... today we rank every...,2024-06-15T20:34:45Z,The Duck,NaN,1,Film & Animation,NaN,swear words a very essential part of the vocab...,None,None,None,None
1,GkQmkZNLCeM,Best Toy Learning Video for Toddlers and Kids ...,Best Toy Learning Video for Toddlers and Kids ...,2021-07-08T18:29:49Z,Genevieve's Playhouse - Learning Videos for Kids,"best, learning video for toddlers, for kids, c...",1,Film & Animation,en,Transcript not available,None,None,None,None
2,_et1i-ykdxA,Best ABC Learning Toy Video for Toddlers! Paw ...,Best ABC Learning Toy Video for Toddlers! Paw ...,2022-07-14T16:45:05Z,Genevieve's Playhouse - Learning Videos for Kids,"abc, for toddlers, for baby, learning, educati...",1,Film & Animation,en,Transcript not available,None,None,None,None
3,_UNe6tz28OI,Best Toy Learning Video for Toddlers and Kids ...,Best Toy Learning Video for Toddlers and Kids ...,2021-03-23T01:43:53Z,Genevieve's Playhouse - Learning Videos for Kids,"best, toddler learning videos, for kids, for b...",1,Film & Animation,en,Transcript not available,None,None,None,None
4,zamt_1PAwqE,The Joy of Sharing Short Episode | Fun Learni...,"Welcome to ""The Learning Show"", the exciting a...",2023-05-19T14:42:38Z,Videogyan Shows - Educational Videos For Kids,"shorts, learning for kids, educational videos,...",1,Film & Animation,en,Transcript not available,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3420,irhVziMbSZk,Moneyball (2011) - Trailer,Real-life story is based on Oakland A's genera...,2014-07-11T03:41:39Z,Sony Pictures at Home UK,"Philip Seymour Hoffman, Brad Pitt, Robin Wrigh...",44,None,NaN,NaN,PT2M29S,126329,0,0
3421,dMaq_pfxs-0,Hitch (2005) - Trailer,"New York ""date doctor"" Alex Hitchens (Will Smi...",2014-02-01T23:07:44Z,Sony Pictures Home Entertainment,"Kevin James, Eva Mendes, Michael Rapaport, Ada...",44,None,en,NaN,PT2M21S,2664971,0,0
3422,Ecn8dvDK7LQ,Lemonade Mouth - Trailer,Be heard. Be strong. Be proud. Its time to tur...,2012-07-11T16:50:42Z,DisneyMoviesOnDemand,"Lemonade Mouth, Bridgit Mendler, Adam Hicks, H...",44,None,NaN,NaN,PT1M33S,1334337,0,0
3423,FQ63rqSRrEI,Kung Fu Panda 2 | Official Trailer,"DreamWorks Animation's Kung Fu Panda 2, in the...",2011-03-05T00:39:13Z,Kung Fu Panda,"YouTube Kids, David Cross, Seth Rogen, Dustin ...",44,None,NaN,guys did you see that I just was like oh I rec...,PT2M14S,2913403,12575,0


In [23]:
# Save the combined dataset to an Excel file
output_file = '../data/raw/combined_dataset.xlsx'
combined_df.to_excel(output_file, index=False, engine='openpyxl')

print(f"Combined dataset saved to {output_file}")

Combined dataset saved to ../data/raw/combined_dataset.xlsx
